Python 提供了几种构建简单类的方式,这些类只是字段的容器,几乎没 有额外功能。这种模式称为“数据类”(data class)

In [1]:
# class Coordinate:

#     def __init__(self,lat,lon):
#         self.lat = lat
#         self.lon = lon

Coordinate类的作用是保存经纬度属性。为` __init__ `方法编写样 板代码容易让人感到枯燥,尤其是属性较多的时候。想想看,每一个属 性都要写 3 次。更糟的是,样板代码并没有给我们提供 Python 对象都 有的基本功能。

In [2]:
# moscow =Coordinate(55.76,37.62)
# moscow# 继承自 object 的 __repr__ 作用不大。

In [3]:
# location = Coordinate(55.76,37.62)
# location==moscow# == 没有意义,因为继承自 object 的 __eq__ 方法比较对象的  ID。

#### 使用 namedtuple 构建 Coordinate 类

In [4]:
from collections import namedtuple
Coordinate = namedtuple('Coordinate','lat lon')
issubclass(Coordinate,tuple)

True

In [5]:
moscow =Coordinate(55.76,37.62)
print(moscow)#有用的 __repr__
location = Coordinate(55.76,37.62)
print(moscow==location)#有意义的 __eq__

Coordinate(lat=55.76, lon=37.62)
True


#### 使用typing.NamedTuple构建Coordinate类

可为每个字段添加类型注解

In [6]:
import typing
Coordinate = typing.NamedTuple('Coordinate', lat=float, lon=float)
issubclass(Coordinate,tuple)

C:\Users\cuter\AppData\Local\Temp\ipykernel_19724\1956521930.py:2: DeprecationWarning: Creating NamedTuple classes using keyword arguments is deprecated and will be disallowed in Python 3.15. Use the class-based or functional syntax instead.
  Coordinate = typing.NamedTuple('Coordinate', lat=float, lon=float)


True

In [7]:
print(typing.get_type_hints(Coordinate))
moscow=Coordinate(55.76,37.62)
moscow

{'lat': <class 'float'>, 'lon': <class 'float'>}


Coordinate(lat=55.76, lon=37.62)

构建带类型的具名元组,也可以通过关键字参数指定字段（要被弃用了QAQ）

这种方式可读性高,而且可以通过映射指定字段及其类型,再使用  **fields_and_types 拆包。

In [8]:
from typing import NamedTuple
class Coordinate(NamedTuple):
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat>=0 else 'S'
        we = 'E' if self.lon>=0 else 'W'
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.lon):.1f}°{we}'

In [9]:
print(typing.get_type_hints(Coordinate))
moscow=Coordinate(55.76,37.62)
print(moscow)
moscow

{'lat': <class 'float'>, 'lon': <class 'float'>}
55.8°N, 37.6°E


Coordinate(lat=55.76, lon=37.62)

In [10]:
lat,lon = moscow
lat,lon

(55.76, 37.62)

dataclass 装饰器读取变量注解,自动为  构建的类生成方法

In [11]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Coordinate:
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat>=0 else 'S'
        we = 'E' if self.lon>=0 else 'W'
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.lon):.1f}°{we}'

`collections.namedtuple` 和 `typing.NamedTuple` 构建的 类是 tuple 的子类,因此实例是不可变的。`@dataclass` 默认构  建可变的类。不过,`@dataclass` 装饰器接受一个关键字参数  `frozen`。指定 `frozen=True`,初始化实例之  后,如果为字段赋值,则抛出异常。

In [12]:
from collections import namedtuple

Point = namedtuple('Point',['x','y'])
p = Point(1,2)
print(p)
print(p.__annotations__)
p.x=1

Point(x=1, y=2)


AttributeError: 'Point' object has no attribute '__annotations__'

In [13]:
from dataclasses import dataclass

@dataclass
class Point:
    x: float=0
    y: float

p = Point(1,2)
p.x = 2
p

TypeError: non-default argument 'y' follows default argument 'x'

只有 typing.NamedTuple 和 dataclass 支持常规的 class 语句句法,方便为构建的类添加方法和文档字符串。

推荐使用 `inspect.get_annotations(MyClass)(Python 3.10 新增)` 或 `typing.get_type_hints(MyClass)`(Python 3.5~3.9)获 取类型信息,因为这两个函数提供了额外的服务,例如可以解析类 型提示中的向前引用

In [14]:
import inspect,typing
print(p.__annotations__)
print(inspect.get_annotations(Point))
typing.get_type_hints(Point)


AttributeError: 'Point' object has no attribute '__annotations__'

构造字典

In [15]:
from collections import namedtuple

Point = namedtuple('Point',['x','y'],defaults=[0])# 不推荐
p = Point(1,2)

p._asdict()

{'x': 1, 'y': 2}

In [16]:
from typing import NamedTuple
class Point1(NamedTuple):
    x: float 
    y: float =0

p1 = Point1(1,2)
p1._asdict()

{'x': 1, 'y': 2}

In [17]:
# from dataclasses import dataclass
import dataclasses

@dataclass
class Point2:
    x: float
    y: float = 0

p2 = Point2(1,2)


print(dataclasses.asdict(p2))
print(p2._asdict())

{'x': 1, 'y': 2}


AttributeError: 'Point2' object has no attribute '_asdict'

获取字段名称和默认值

In [18]:
print(dataclasses.fields(p2))
print(p._fields)
print(p1._fields)
print([f.name for f in dataclasses.fields(p2)])

print(p._field_defaults)
print(p1._field_defaults)
print([{f.name, f.default} for f in dataclasses.fields(p2)
        if isinstance(f.default,(int, float))])

(Field(name='x',type=<class 'float'>,default=<dataclasses._MISSING_TYPE object at 0x0000024AC95E5BE0>,default_factory=<dataclasses._MISSING_TYPE object at 0x0000024AC95E5BE0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD), Field(name='y',type=<class 'float'>,default=0,default_factory=<dataclasses._MISSING_TYPE object at 0x0000024AC95E5BE0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD))
('x', 'y')
('x', 'y')
['x', 'y']
{'y': 0}
{'y': 0}
[{0, 'y'}]


更改之后创建新实例

In [19]:
mp=p._replace(x=2)
print(mp)
mp1=p1._replace(x=2)
print(mp1)
mp2=dataclasses.replace(p2,x=2)
print(mp2)
p2.x=3

p2

Point(x=2, y=2)
Point1(x=2, y=2)
Point2(x=2, y=2)


Point2(x=3, y=2)

`collections.namedtuple` 是一个工厂函数,用于构建增强的  tuple 子类,具有字段名称、类名和提供有用信息的 `__repr__` 方  法。

创建具名元组需要指定两个参数:一个类名和一个字段名称列表。后 一个参数可以是产生字符串的可迭代对象,也可以是一整个以空格分隔 的字符串。

In [20]:
from collections import namedtuple

City = namedtuple('City','name country population coordinates')
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
# 根据可迭代对象构建 City 实例
Coordinate = namedtuple('Coordinate', 'lat lon')
delhi_data = ('Delhi NCR', 'IN', 21.935, Coordinate(28.613889, 77.208889))
delhi = City._make(delhi_data)
delhi._asdict()
# ._asdict() 方法可把数据序列化成 JSON 格式。
import json
json.dumps(delhi._asdict())

'{"name": "Delhi NCR", "country": "IN", "population": 21.935, "coordinates": [28.613889, 77.208889]}'

变量注解的意义

类型提示在运行时没有作用。然而,Python 在导入时 (加载模块时)会读取类型提示,构建 `__annotations__` 字典,供  `typing.NamedTuple` 和 `@dataclass` 使用,增强类的功能。

In [21]:
class DemoPlainClass:
    a:int # a 出现在 __annotations__ 中,但被抛弃了,因为该类没有名为  a 的属性。
    # a 只作为注解存在,不是类属性,因为没有绑定值
    b:float = 1.1
    c = 'spam' # c 是普通的类属性,没有注解。

print(DemoPlainClass.__annotations__)
print(DemoPlainClass.a)


{'a': <class 'int'>, 'b': <class 'float'>}


AttributeError: type object 'DemoPlainClass' has no attribute 'a'

In [22]:
class DemoNTClass(NamedTuple):
    a:int #a 是注解,也是实例属性
    b:float = 1.1
    c = 'spam' 

print(DemoNTClass.__annotations__)
print(DemoNTClass.a)
print(DemoNTClass.b)
print(DemoNTClass.c)

{'a': <class 'int'>, 'b': <class 'float'>}
_tuplegetter(0, 'Alias for field number 0')
_tuplegetter(1, 'Alias for field number 1')
spam


In [23]:
from dataclasses import dataclass

@dataclass()
class DemoDataClass:
    a: int #是注解,也是受描述符控制的实例属性。
    b: float =1.1# b 是注解,也是受描述符控制的实例属性,默认值为  1.1
    c = 'spam'

print(DemoDataClass.__annotations__)
print(DemoDataClass.__doc__)
# print(DemoDataClass.a)

{'a': <class 'int'>, 'b': <class 'float'>}
DemoDataClass(a: int, b: float = 1.1)


dataclass实例可变，还可为不存在的属性赋值

In [24]:
dc = DemoDataClass(9)
dc.c = 'whatever'
dc.z = 'sectet'# 实例属性
dataclasses.asdict(dc)
dc.z



'sectet'

@dataclass关键字参数
- `init`: 生成`__init__`
- `repr`:生成`__repr__`
- `eq`:生成`__eq__`
- `order`:生成`__lt__`、`__le__`、`__gt__`、`__ge__`
- `unsafe_hash`:生成`__hash__`
- `frozen`:控制实例是否可变

如果 `eq` 和 `frozen` 参数的值都是 `True`,那么 `@dataclass` 将生成 一个合适的 `__hash__` 方法,确保实例是可哈希的。生成的  `__hash__` 方法使用所有字段的数据,通过字段选项也不能排除。对于 `frozen=False`(默认值),@dataclass 把 `__hash__` 设为 None,覆盖从任何超类继承的 `__hash__` 方法,表 明实例不可哈希。

### 字段选项

在提供类型提示的同时设定默认 值。声明的字段将作为参数传给生成的 `__init__` 方法。Python 规  定,带默认值的参数后面不能有不带默认值的参数。因此,为一个字段 声明默认值之后,余下的字段都要有默认值。

类属性通常用作 实例属性的默认值,数据类也是如此。`@dataclass` 使用类型提示中的  默认值生成传给 `__init__` 方法的参数默认值。

dataclasses 模块中的一个安全特性，防止了一个常见的编程陷阱：在 Python 中，默认参数值只在函数（或类）定义时计算一次，而不是每次创建实例时都重新计算。这意味着所有 Event 实例都会共享同一个列表对象

In [25]:
@dataclass
class ClubMember:
    name: str
    guests: list = []
    

ValueError: mutable default <class 'list'> for field guests is not allowed: use default_factory

default_factory 参数的值可以是一个函数、一个类,或者其他可调  用对象,在每次创建数据类的实例时调用(不带参数),构建默认值。 这样,每个 ClubMember 实例都有自己的一个 list,而不是所有实  例共用同一个 list。

In [26]:
from dataclasses import dataclass, field

@dataclass
class ClubMember:
    name: str
    guest: list = field(default_factory=list)
    

In [27]:
class Test:
    name: str = 'apple'
    lis: list[str] = []

fruit1 = Test()
fruit1.name+='a'# 重新赋值，创建实例属性
print(fruit1.name)
fruit2=Test()
print(fruit2.name)
fruit2.name = 'banana'# 赋值，创建实例属性
fruit2.name
fruit2.lis.append('a')# 先找实例属性，由于没有，所以再找类属性，在类属性上修该
fruit1.lis=['a','b']# 赋值，创建实例属性
print(fruit1.lis)
print(fruit2.lis)
print(Test.lis)



applea
apple
['a', 'b']
['a']
['a']


In [28]:
@dataclass
class Test:
    lis: list = ['1','2']

ValueError: mutable default <class 'list'> for field lis is not allowed: use default_factory

In [29]:
@dataclass
class Test:
    lis: list = field(default_factory=lambda: ['1'])

t1 = Test()
t1.lis

['1']

field函数接受的关键字参数
- default
- default_factory
- init
- repr
- compare
- hash

In [30]:
@dataclass
class ClubMember: 
    name: str= field(default='hana',init=True)
    guests: list = field(default_factory=list) 
    athlete: bool = field(default=False, repr=False)

    # def __repr__(self):
    #     return f'{self.name} is athlete = {self.athlete}'

geust1 = ClubMember()
# geust2 = ClubMember(name='hana')

geust1.name = 'bana'
geust2.name

NameError: name 'geust2' is not defined

@dataclass 生成的 `__init__` 方法只做一件事:把传入的参数及其  默认值(如未指定值)赋值给实例属性,变成实例字段。可是,有些时 候初始化实例要做的不只是这些。为此,可以提供一个  `__post_init__` 方法。如果存在这个方法,则 @dataclass 将在生 成的 `__init__` 方法最后调用 `__post_init__` 方法。 `__post_init__` 经常用于执行验证,以及根据其他字段计算一个字段  的值。下面举个例子说明这两种用途。

在dataclass中，当你使用类型注解并赋值时，Python会将其视为实例字段而不是类字段。这意味着：

- 每个实例会有自己的all_handle副本，而不是共享同一个
- 这违背了你"目的就是要共享"的初衷
- 如果你尝试通过类名访问它，可能会遇到问题

若想为其指定类型，则要使用ClassVar()

In [ ]:
'''
HackerClubMember构造函数接受一个可选的handle参数
'''
from typing import ClassVar

@dataclass
class HackerClubMember(ClubMember):
    all_handle: ClassVar[set[str]]= set()# 目的就是要共享
    handle: str = ''

    def __post_init__(self):
        cls = self.__class__#获取实例所属的类
        if self.handle == '':
            self.handle = self.name.split()[0]
        if self.handle in cls.all_handle:
            msg = f'handle {self.handle} already exists.'
            raise ValueError
        cls.all_handle.add(self.handle)




@dataclass 装饰器不关心注解中的类型,但有两种例外情况,这是其  中之一,即类型为 ClassVar 时,不为属性生成实例字段。  另外一种情况是声明“仅作初始化的变量”

这是一种特殊的参数，你会在类的 `__init__ `方法中用到它来帮助设置其他字段的值，但这个参数本身不会被存储为对象的一个属性。

In [33]:
from dataclasses import InitVar
@dataclass
class C:
    i: int
    j: int = None
    database: InitVar[DatabaseType] = None

    def __post__init__(self,database):
        if self.j is None and database is not None:
            self.j = database.lookup('j')#InitVar 参数的生命周期在 __post_init__ 方法结束时终止，它们不会被保存到实例中。



NameError: name 'DatabaseType' is not defined

In [39]:
from dataclasses import dataclass, field,fields
from typing import Optional
from enum import Enum, auto
from datetime import date

class ResourceType(Enum):
    BOOK = auto()
    EBOOK = auto()
    VIDEO = auto()
    
@dataclass
class Resource:
    '''描述媒体资源'''
    identifier: str
    title: str = '<untitled>'
    creators: list[str] = field(default_factory=list)
    date: Optional[date] = None
    type: ResourceType = ResourceType.BOOK
    description: str =''
    language: str =''
    subjects: list[str] = field(default_factory=list)

    def __repr__(self):
        cls = self.__class__
        cls_name = cls.__name__
        indent = ' '*4
        res = [f'{cls_name}(']
        for f in fields(cls):
            value = getattr(self,f.name)
            res.append(f'{indent}{f.name}={value!r}')
        res.append(')')
        return "\n".join(res)
        

In [40]:
description = 'Improving the design of existing code'
book = Resource('978-0-13-475759-9', 'Refactoring, 2nd Edition',
                ['Martin Fowler', 'Kent Beck'], date(2018, 11, 19),
                ResourceType.BOOK, description, 'EN',
                ['computer programming', 'OOP'])

book

Resource(
    identifier='978-0-13-475759-9'
    title='Refactoring, 2nd Edition'
    creators=['Martin Fowler', 'Kent Beck']
    date=datetime.date(2018, 11, 19)
    type=<ResourceType.BOOK: 1>
    description='Improving the design of existing code'
    language='EN'
    subjects=['computer programming', 'OOP']
)

类模式通过类型和属性(可选)匹配类实例。类模式的匹配对象可以是 任何类的实例,而不仅仅是数据类的实例

类模式是 match...case 语句中的一种模式，用于检查一个对象是否属于某个类（或其子类），并且可选地将其属性与子模式进行匹配。它的语法看起来非常像调用类的构造函数。

In [ ]:
match x:
    case float():
        do ....

关键字类模式


In [44]:
import typing

class City(typing.NamedTuple): 
    continent: str 
    name: str 
    country: str 
    
cities = [
    City('Asia', 'Tokyo', 'JP'), 
    City('Asia', 'Delhi', 'IN'), 
    City('North America', 'Mexico City', 'MX'), 
    City('North America', 'New York', 'US'), 
    City('South America', 'São Paulo', 'BR'),]

In [45]:
def match_asian_countries(): 
    results = [] 
    for city in cities: 
        match city: 
            case City(continent='Asia', country=cc): 
                results.append(cc)
    return results

位置类模式

City 或其他类若想使用位置模式,要有一个名为 `__match_args__ ` 的特殊类属性。本章讲到的类构建器会自动创建这个属性。对于 City  类,`__match_args__` 属性的值如下所示。
```
>>> City.__match_args__
('continent', 'name', 'country')
```

In [ ]:
def match_asian_countries_pos(): 
    results = [] 
    for city in cities: 
        match city: 
            case City('Asia', _, country): 
                results.append(country)

    return results